In [ ]:
import tensorflow as tf
import numpy as np
import re

In [ ]:
dataset = open("corpus.txt", "r", encoding = "utf-8").read()

In [ ]:
dataset[:100]

'In the Devoid of Sorrow\nThe Air and my Heart full of Luster\nBehind the Mountains\nThere must be a key'

NEWLINE PRESERVATION

In [ ]:
# IMPLEMENT ON THE DATASET

In [ ]:
test = "In the Devoid of Sorrow\nThe Air and my Heart full of Luster\nBehind the Mountains\nThere must be a key"

In [ ]:
test = re.sub(r"[\n]", " newline ", test)

In [ ]:
test.split()

In [ ]:
# FOR LATER
# test = re.sub(r"newline", "\n", test)

PREPROCESSING

In [ ]:
tokens = dataset.split()

In [ ]:
tokens[:10]

In [ ]:
def clean_text(text):
    # punctuation, except ' and -
    text = re.sub(r"[^a-zA-Z'-]", " ", text)
    # double whitespaces
    text = re.sub(r" +", " ", text)
    # lower-case
    text = text.lower()

# ADD SPECIAL CASES AND ARCHAISMS
  
    return text

In [ ]:
text = [clean_text(token) for token in tokens]

In [ ]:
text[:10]

EXPLORATORY DATA ANALYSIS

In [ ]:
# total tokens
len(tokens)

In [ ]:
# unique tokens
len(set(tokens))

SEQUENCES

In [ ]:
seq_len = 50 + 1
sequences = []

for i in range(seq_len, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-seq_len:i]
	# convert into a line
	line = ' '.join(seq)
 
	sequences.append(line)

In [ ]:
sequences[:20]

In [ ]:
len(sequences)

331252

In [ ]:
# SAVE NEW DOC

In [ ]:
# CREATE TEST SET

MODEL

MODEL TRAINING

GENERATION